In [1]:
from transformers import pipeline

import numpy as np
import pandas as pd

from google.colab import userdata

In [2]:
from huggingface_hub import login

HF = userdata.get('HF_TOKEN')
login(HF)

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manashjyotiborah/top-10000-movies-hosted-on-tmdb")

print("Path to dataset files:", path)

100%|██████████| 16.9M/16.9M [00:01<00:00, 9.54MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/manashjyotiborah/top-10000-movies-hosted-on-tmdb/versions/2


In [4]:
keywords_df = pd.read_csv(f'{path}/keywords_dataframe.csv', index_col='id')
mov_df = pd.read_csv(f'{path}/movies_dataset.csv', index_col='id')

keywords_df.drop(['Unnamed: 0'], axis=1, inplace=True)

movies_df = mov_df.merge(keywords_df, on='id')
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

movies_df.head()

,genre_ids,original_language,overview,popularity,release_date,title,vote_average,vote_count,genres,keywords
id,,,,,,,,,,
385687,"[28, 80, 53]",en,Over many missions and against impossible odds...,4654.279,2023-05-17,Fast X,7.3,2093,"Action, Crime, Thriller","[{'id': 9663, 'name': 'sequel'}, {'id': 9748, ..."
697843,"[28, 53]",en,Tasked with extracting a family who is at the ...,2494.010,2023-06-09,Extraction 2,7.7,910,"Action, Thriller","[{'id': 3070, 'name': 'mercenary'}, {'id': 966..."
603692,"[28, 53, 80]",en,"With the price on his head ever increasing, Jo...",1920.127,2023-03-22,John Wick: Chapter 4,7.9,3344,"Action, Thriller, Crime","[{'id': 242, 'name': 'new york city'}, {'id': ..."
569094,"[28, 12, 16, 878]",en,"After reuniting with Gwen Stacy, Brooklyn’s fu...",2013.795,2023-05-31,Spider-Man: Across the Spider-Verse,8.6,1796,"Action, Adventure, Animation, Science Fiction","[{'id': 2858, 'name': 'sacrifice'}, {'id': 328..."
502356,"[16, 10751, 12, 14, 35]",en,"While working underground to fix a water main,...",1539.037,2023-04-05,The Super Mario Bros. Movie,7.8,5165,"Animation, Family, Adventure, Fantasy, Comedy","[{'id': 282, 'name': 'video game'}, {'id': 690..."


In [5]:
movies_df.dropna(axis=0, inplace=True)

In [ ]:
movies_df['overview'].iloc[0]

"Over many missions and against impossible odds, Dom Toretto and his family have outsmarted, out-nerved and outdriven every foe in their path. Now, they confront the most lethal opponent they've ever faced: A terrifying threat emerging from the shadows of the past who's fueled by blood revenge, and who is determined to shatter this family and destroy everything—and everyone—that Dom loves, forever."

In [6]:
genres_list = list(movies_df['genres'].unique())
gl = []
for g in genres_list:
    try:
        gl.extend(g.split(', '))
    except:
        continue

GENRES = list(set(gl))
GENRES

['Western',
 'Romance',
 'Mystery',
 'Adventure',
 'Thriller',
 'Fantasy',
 'History',
 'Drama',
 'Action',
 'Family',
 'Animation',
 'Comedy',
 'Documentary',
 'Crime',
 'Music',
 'TV Movie',
 'Horror',
 'War',
 'Science Fiction']

In [7]:
data = movies_df[['overview', 'genres']].copy()

In [8]:
classifier = pipeline("zero-shot-classification")
result = classifier(
    data['overview'].to_list()[:100],
    candidate_labels=GENRES,
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import pickle

with open('result.pickle', 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [26]:
result[0]

{'sequence': "Over many missions and against impossible odds, Dom Toretto and his family have outsmarted, out-nerved and outdriven every foe in their path. Now, they confront the most lethal opponent they've ever faced: A terrifying threat emerging from the shadows of the past who's fueled by blood revenge, and who is determined to shatter this family and destroy everything—and everyone—that Dom loves, forever.",
 'labels': ['Family',
  'Adventure',
  'Crime',
  'Drama',
  'Documentary',
  'War',
  'Mystery',
  'Western',
  'Action',
  'History',
  'Thriller',
  'Romance',
  'Horror',
  'Music',
  'TV Movie',
  'Animation',
  'Fantasy',
  'Science Fiction',
  'Comedy'],
 'scores': [0.539378821849823,
  0.05868273228406906,
  0.03785441070795059,
  0.036856021732091904,
  0.03238692879676819,
  0.029787635430693626,
  0.02949064038693905,
  0.027093760669231415,
  0.026329144835472107,
  0.025958452373743057,
  0.02471137046813965,
  0.024462483823299408,
  0.021078282967209816,
  0.017

In [47]:
from collections import Counter

predicts = []
for res in result:
    r = {key: value for key, value in zip(res['labels'], res['scores'])}

    c = Counter(r)
    most_common = c.most_common(3)
    max_values_keys = [key for key, val in most_common]
    predicts.append(max_values_keys)


In [50]:
data = movies_df[['title', 'overview', 'genres']].iloc[:100].copy()

data['predicts'] = pd.Series(predicts, index=data.index)
data.head()

,title,overview,genres,predicts
id,,,,
385687,Fast X,Over many missions and against impossible odds...,"Action, Crime, Thriller","[Family, Adventure, Crime]"
697843,Extraction 2,Tasked with extracting a family who is at the ...,"Action, Thriller","[Family, Crime, Action]"
603692,John Wick: Chapter 4,"With the price on his head ever increasing, Jo...","Action, Thriller, Crime","[Adventure, Crime, Action]"
569094,Spider-Man: Across the Spider-Verse,"After reuniting with Gwen Stacy, Brooklyn’s fu...","Action, Adventure, Animation, Science Fiction","[Adventure, Drama, Action]"
502356,The Super Mario Bros. Movie,"While working underground to fix a water main,...","Animation, Family, Adventure, Fantasy, Comedy","[Mystery, Adventure, Family]"
